In [94]:
import math
import numpy as np

# LOX Tank Volume Calculator

In [95]:
burntime=10 # seconds
massflow_pressurant=1.7779 #kg/s
OF=2.2 # Oxidizer to Fuel ratio
ullageRatio=1.15 # percentage of tank that is ullage
#T= # in Kelvin
density=1097.26 #kg/m^3

def loxVolume(burntime, massflow_pressurant, OF, ullageRatio):
    massflow=massflow_pressurant*OF/(OF+1)
    mass=burntime*massflow
    volume=mass/density
    volumeFinal=volume*ullageRatio
    print("lox massflow = " + str(massflow) + " kg/s")
    print("lox massflow = " + str(massflow*2.20462) + " lb/s")
    print("lox mass = " + str(mass) + " kg")
    print("lox mass = " + str(mass*2.20462) + " lb")
    print("lox tank volume = " + str(volumeFinal) + " m^3")
    print("lox tank volume = " + str(volumeFinal*35.3147) + " ft^3")
    

loxVolume(burntime,massflow_pressurant,OF,ullageRatio)

lox massflow = 1.22230625 kg/s
lox massflow = 2.6947208048749998 lb/s
lox mass = 12.2230625 kg
lox mass = 26.947208048749996 lb
lox tank volume = 0.012810566205821772 m^3
lox tank volume = 0.45240130238873416 ft^3


# Kerosene Tank Volume Calculator

In [96]:
burntime=10 # seconds
massflow_pressurant=1.77 #kg/s
OF=2.2 # Oxidizer to Fuel ratio
ullageRatio=1.15 # percentage of tank that is ullage
#T= # in Kelvin
density=820 #kg/m^3

def keroVolume(burntime, massflow_pressurant, OF, ullageRatio):
    massflow=massflow_pressurant*1/(OF+1)
    mass=burntime*massflow
    volume=mass/density
    volumeFinal=volume*ullageRatio
    print("kerosene massflow = " + str(massflow) + " kg/s")
    print("kerosene massflow = " + str(massflow*2.20462) + " kg/s")
    print("kerosene mass = " + str(mass) + " kg")
    print("kerosene mass = " + str(mass*2.20462) + " lb")
    print("kerosene tank volume = " + str(volumeFinal) + " m^3")
    print("kerosene tank volume = " + str(volumeFinal*35.3147) + " ft^3")
    
keroVolume(burntime,massflow_pressurant,OF,ullageRatio)

kerosene massflow = 0.553125 kg/s
kerosene massflow = 1.2194304374999998 kg/s
kerosene mass = 5.53125 kg
kerosene mass = 12.194304375 lb
kerosene tank volume = 0.007757240853658536 m^3
kerosene tank volume = 0.2739446335746951 ft^3


# Tank Height Needed to House Propellant (With Ellipsoidal Ends)

In [97]:
semi_axis=1 #in
radius=3 #in
propVolume=0.2739 #ft^3 (ullage ratio factored in from keroVolume function)

def elpsTankHeight(semi_axis,radius, propVolume):
    propVolume=propVolume*12**3 #ft^3 to in^3
    print("propellant tank volume " + str(propVolume) + " in^3")
    elpsCapVolume=2*2/3*math.pi*radius**2*semi_axis #volume of both caps combined
    print("volume of ellipsoidal caps = " + str(elpsCapVolume) + " in^3")
    elpsTankHeight=(propVolume - elpsCapVolume)/math.pi/radius**2 #solve for tank height in between caps
    print("cylinder tank height for 6 in diameter with " + str(semi_axis) + " in ellipsoidal caps is: " + str(elpsTankHeight) + " in")

elpsTankHeight(semi_axis, radius, propVolume)

propellant tank volume 473.2992 in^3
volume of ellipsoidal caps = 37.69911184307752 in^3
cylinder tank height for 6 in diameter with 1 in ellipsoidal caps is: 15.406201609208798 in


# Pressurant Tank Volume Calculator

In [98]:
burntime=10 # seconds
massflow_pressurant=1.7779 #kg/s
OF=2.2 # Oxidizer to Fuel ratio
ullageRatio=1.15 # percentage of tank that is ullage
T=298 # K (298K is room temperature)
P=np.linspace(2000,4000,5) # psi range of pressures
density=1097.26 #kg/m^3

def calculate_compressibility_factor(temperature, pressure):
    # Constants for nitrogen (N2)
    critical_temperature = 126.2  # K
    critical_pressure = 3.39  # MPa
    pressure=pressure*6894.76/1e6 # convert from psi

    # Redlich-Kwong parameters
    a = 0.42748 * (critical_temperature ** 2.5) / critical_pressure
    b = 0.08664 * critical_temperature / critical_pressure

    # Calculate A and B terms in the Redlich-Kwong equation
    A = a * pressure / (8.314 * temperature ** 2)
    B = b * pressure / (8.314 * temperature)

    n=len(pressure)
    Z=np.zeros(n)

    # Solve the cubic equation to find the compressibility factor
    for i in range(0,len(pressure)):
        coefficients = [1, -1, A[i] - B[i] - B[i]**2, -A[i] * B[i]]
        roots = np.roots(coefficients)
        
        # Filter the real and positive roots (physical solutions)
        valid_roots = [root for root in roots if root.real > 0 and root.imag == 0]

        if not valid_roots:
            return None
        else:
            Z[i] = max(valid_roots)
        
    return Z

def calculate_nitrogen_volume(massflow_pressurant, burntime, T, P, Z):
    molar_mass_nitrogen = 0.0280134  # kg/mol
    R = 8.314  # J/(mol·K)

    P=P*6894.76 # convert from psi to Pa

    mass = massflow_pressurant*burntime

    moles_of_nitrogen = mass / molar_mass_nitrogen

    volume = (Z * moles_of_nitrogen * R * T) / P

    volume = volume * 35.3147 # convert from m^3 to ft^3

    return volume

Z=calculate_compressibility_factor(T,P)
n2Volume=calculate_nitrogen_volume(massflow_pressurant, burntime, T, P, Z)
for i in range(0,len(n2Volume)):
    print(str(Z[i]) + " Compressibility factor for N2 at temperature of " + str(T) + " K and pressure of " + str(P[i]) + "psi")
    print('\n')
    print(str(n2Volume[i]) + " ft^3 Volume N2 for temperature of " + str(T) + " K and pressure of " + str(P[i]) + "psi")
    print('\n')


0.019710041005212245 Compressibility factor for N2 at temperature of 298 K and pressure of 2000.0psi


0.07937097067984308 ft^3 Volume N2 for temperature of 298 K and pressure of 2000.0psi


0.024636433824419532 Compressibility factor for N2 at temperature of 298 K and pressure of 2500.0psi


0.07936737082248767 ft^3 Volume N2 for temperature of 298 K and pressure of 2500.0psi


0.02956238122436565 Compressibility factor for N2 at temperature of 298 K and pressure of 3000.0psi


0.0793637751358973 ft^3 Volume N2 for temperature of 298 K and pressure of 3000.0psi


0.034487883978340586 Compressibility factor for N2 at temperature of 298 K and pressure of 3500.0psi


0.07936018361202139 ft^3 Volume N2 for temperature of 298 K and pressure of 3500.0psi


0.039412942857646405 Compressibility factor for N2 at temperature of 298 K and pressure of 4000.0psi


0.07935659624283206 ft^3 Volume N2 for temperature of 298 K and pressure of 4000.0psi




C:\Users\jared\AppData\Local\Temp\ipykernel_18848\2163250468.py:37: ComplexWarning: Casting complex values to real discards the imaginary part
  Z[i] = max(valid_roots)


## Inputs
Pressurant \
Burn Time \
Mass Flow \
Propellants \
\
Assuptions: \
Isothermal, Isenthalpic, adiabatic expansion \ 
Ideal \
Compressibility factor \
\
Based on the assumption, only the output will be changed based on which values satisfy an equation within tolerance
\
Pressurant (N2, H) \
OF ratio \
\
Propellants, only two temperatures needed \
incompressible \
\
Pressurant starts room temperature gets colder through adiabatic (assuming no energy leaves system) expansion. function of mass flow. Convert to volume increase to figure out temperature change. Temperature change will affect pressure in N2 line? Research needed.\

## Outputs
Propellant tank volume \

